In [52]:
# imports
# system related
import sys
import subprocess
from os import path

# basics
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from time import time
import datetime
from time import sleep

# snap7 and logging
import snap7 as s7
import csv



print('imports succesful')

imports succesful


## Connecting to plc

In [53]:
# connecting to plc

plc = s7.client.Client()
plc.connect('192.168.0.19', 0, 1, )  # ip, rack, slot

print(f'connection status: {plc.get_connected()}')

connection status: True


## Defining get & set functions for plc variables

In [54]:
# Defining read/write functions

def get_bool(db_number, start_offset, bit_offset=0):
    data = plc.db_read(db_number, start_offset, 1) # read 1 bite
    value = s7.util.get_bool(data, 0, bit_offset)
    # print(f'db number: {db_number}, bit: {start_offset}, has boolean value: {value}')
    return value
    
def set_bool(db_number, start_offset, value, bit_offset=0):
    data = plc.db_read(db_number, start_offset, 1) # read 1 bite
    s7.util.set_bool(data, 0, bit_offset, value)
    plc.db_write(db_number, start_offset, data)
    return None


def get_real(db_number, start_offset):
    data =  plc.db_read(db_number, start_offset, 4) # read 4 bites
    real = s7.util.get_real(data, 0)
    return real

def set_real(db_number, start_offset, value):
    data =  plc.db_read(db_number, start_offset, 4) # read 4 bites
    s7.util.set_real(data, 0, value)
    plc.db_write(db_number, start_offset, data)
    return None



    


# start make datalog function for ixon db

In [59]:

def log_loop(csvfile):
    # whole_start = datetime.datetime.now()
        
    # defining constants
    # db number
    db_log = 33
    
    # start offsets
    log_so_width  = 0
    log_so_rpm    = 4
    log_so_rpm_sp = 8
    log_so_torque = 12
    log_so_hz1    = 16
    log_so_hz1_sp = 20
    log_so_hz2    = 24
    log_so_hz2_sp = 28
    log_so_hz3    = 32
    log_so_hz3_sp = 36
    log_so_hz4    = 40
    log_so_hz4_sp = 44
    
    cols = ['width', 'rpm', 'rpm_sp', 'torque', 'hz1', 'hz1_sp', 'hz2', 'hz2_sp', 'hz3', 'hz3_sp', 'hz4', 'hz4_sp']

    dt = 0.050  # 50 ms = 20 times / s
    new_log_time = np.ceil(time())
    
    # write headers to the .csv file
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['time'] + cols)
    
    print('log sequence initialized')

    try:
        print('logging has started')
        while True:
            
            # waiting untill enough time has elapst
            while time() < new_log_time:
                pass
            # adjust time to next one to log to
            new_log_time = np.round(new_log_time + dt, 5)
            
            # read DB
            data =  plc.db_read(db_log, 0, 48)
            
            # converting bytes to reals
            vals = [new_log_time]
            for i, name in enumerate( cols):
                val = s7.util.get_real(data, 4*i)
                vals.append(val)
            
            # write to .csv file
            csv_writer.writerow(vals)
            csvfile.flush()   ####### important, write while loop is running!!
            
    except KeyboardInterrupt:
        print(f'logging ended by keystroke. Thank you for choosing YoussefLog services.') 

name = f'../data/petg30gs-up_down-{datetime.date.today()}'  #### CHANGE TO DATA FOLDER
if path.isfile(name+'.csv'): 
    for i in range(1, 100):
        if not path.isfile(name + f'({i}).csv'):
            name = name + f'({i})'
            break
    
with open(name + '.csv', 'w') as csvfile:
    print(f'.csv file created')
    log_loop(csvfile)

.csv file created
log sequence initialized
logging has started
logging ended by keystroke. Thank you for choosing YoussefLog services.


### another way of unpacking a database data (Thijs)

In [43]:
import struct
# struct.unpack("b",plc.db_read(db_number, 2, 1)) 

In [32]:
%%time
for i in  range(1000):
    plc.db_read(33, 0, 48)

b'CPU : Address out of range'


RuntimeError: b'CPU : Address out of range'

12.3 ns ± 1.38 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


CPU times: total: 0 ns
Wall time: 0 ns


2

In [46]:
%%time
time()

CPU times: total: 0 ns
Wall time: 0 ns


1667382946.0825675

0.010970830917358398